In [17]:
import json 

# Load the emotions from the JSON file
with open('./data/emotions.json', 'r') as f:
    emotions = json.load(f)

#convert the emotions.positive to a list in a new variable
emotions_list = '\n'.join([str(elem)for i, elem in enumerate (emotions['positive'])])
emotions_list = emotions_list + '\n'.join([str(elem)for i, elem in enumerate (emotions['negative'])])
emotions_list = emotions_list + '\n'.join([str(elem)for i, elem in enumerate (emotions['complex'])])

print(emotions_list)

Joyful
Grateful
Serene
Interested
Hopeful
Proud
Amusement
Inspired
Awed
LovedAngry
Sad
Afraid
Disgusted
Ashamed
Guilty
Envious
Jealous
Frustrated
LonelyNostalgic
Empathetic
Sympathetic
Content
Satisfied
Confused
Anticipative
Relieved


In [18]:
# import langchain modules for ollama
from langchain_community.llms import Ollama

llm = Ollama(model="mistral")
llm.invoke("Are you listening to me?")


" Yes, I am designed to listen and process text-based inputs. However, please keep in mind that I don't have the ability to understand context or emotions in the way humans do. I can only provide responses based on the information provided in the text. If you have any specific questions or topics you'd like me to help with, feel free to ask!"

In [52]:
# import langchain prompttemplate
from langchain.prompts import PromptTemplate

# define the template
prompt_template = """
You are PHD candidate in the psychology department. You have been given the task of labeling text from students to determine their emotional state.
You will be given a sentence or two from the student. You will attempt to infer the students emotional state from the text. 

You will limit your the emotions to the following list of emotions:
{emotion_list}

Your task is to provide a single word that best describes the emotional state from the first person perspective.
Sad
Happy
Angry

If you believe there could be multiple emotions, choose the one that is most prevalent. 
If the text refers to multiple people, you will only be given the emotion from the perspective of the first person. Choose the best emotion for the first person. 
There is not penalty for providing a wrong answer.

The students text:
{text}

Remember to only respond with a single word. Do not explain your reasoning. 1 word response only. Such as "Happy", "Sad" or "Angry"


SINGLE 1 WORD ANSWER ONLY.
"""

prompt = PromptTemplate.from_template(prompt_template)
prompt_formatted_str: str = prompt.format(
    emotion_list=emotions_list, text="i was feeling at the start didnt want to move much at all was really glad to experience this glimpse into the sort of vibrant energy i will gain through out the year"
)

llm = Ollama(model="mistral")
prediction = llm.predict(prompt_formatted_str)
print(prediction)


    

 Grateful. The student expresses feeling "glad" to have experienced something, indicating a positive emotional state. However, the use of the word "glimpse" and "vibrant energy I will gain throughout the year" suggests that this moment was significant for the student and they are grateful for it.


In [59]:
# import langchain prompttemplate
from langchain.prompts import PromptTemplate

# define the template
prompt_template = """
As a psychology Ph.D. candidate, your task is to identify the predominant emotional state of a student based on their provided text. 
You should limit your responses to one of the following emotions: {emotion_list} Ensure your response reflects the emotional state 
from the first person's perspective. Respond with only one word that best captures the emotion. Do not provide explanations. 
Use just one of the specified emotions as your answer.

Students' text: {text}

Response: [Your one-word emotion here]
"""

prompt = PromptTemplate.from_template(prompt_template)
prompt_formatted_str: str = prompt.format(
    emotion_list=emotions_list, text="i was feeling at the start didnt want to move much at all was really glad to experience this glimpse into the sort of vibrant energy i will gain through out the year"
)

llm = Ollama(model="gemma")
prediction = llm.predict(prompt_formatted_str)
print(prediction)


    

Hopeful


In [60]:
# let's make the above into a function called predict_emotion
def predict_emotion(text):  
    prompt = PromptTemplate.from_template(prompt_template)
    prompt_formatted_str: str = prompt.format(
        emotion_list=emotions_list, text=text
    )

    llm = Ollama(model="mistral")
    prediction = llm.predict(prompt_formatted_str)
    return prediction


In [30]:
!pip install pandas

  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 23.2 MB/s eta 0:00:0000:0100:01
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)


In [69]:
# read in the ./data/emotions_sentiment_sample.csv file using pandas and send the text column to the predict_emotion function
import pandas as pd

df = pd.read_csv('./data/emotions_sentiment_sample.csv')
#loop through the text column and call the predict_emotion function
for text in df['text']:
    prediction = predict_emotion(text)
    #check to see if there is more than one word in prediction, if so, split and take the first word only 
    if len(prediction.split()) > 1:
        prediction = prediction.split()[0]
    
    #strip any whitespace from the prediction or punctuation from prediction
    prediction = prediction.strip().strip('.,')

    print(text + " : " + prediction)



i seriously hate one subject to death but now i feel reluctant to drop it : Confused
im so full of life i feel appalled : Angered
i sit here to write i start to dig out my feelings and i think that i am afraid to accept the possibility that he might not make it : Afraid
ive been really angry with r and i feel like an idiot for trusting him in the first place : Angry
i feel suspicious if there is no one outside like the rapture has happened or something : Fear
i feel jealous becasue i wanted that kind of love the true connection between two souls and i wanted that : Jealous
when a friend of mine keeps telling me morbid things that happened to his dog : Sad
i finally fell asleep feeling angry useless and still full of anxiety : Angry
i feel a bit annoyed and antsy in a good way : Frustrated
i feel like i ve regained another vital part of my life which is living : Joyful
i feel a bit like franz liebkind in the producers not many people know it but the fuhrer was a terrific dancer : Amused